## SPlot fit to invariant mass of 2 kaons

The sPlot technique is used to perform background subtraction. Here we assume signal is just phi photoproduction, while background can be other 2 kaon final states or mis-identified events.

We use simulated data to provide a good approximation of the phi PDF including experimental resultions and acceptances.

The background is assumed to be smooth under the phi peak and is paramterised by a 2 parameter Chebychev.

The classes used here are wrappers to the RooStats sPlot implementation, which uses a RooFit extended maximum likelihood fit to the data to determine signal and background yeilds and covariances.

In [1]:
gROOT->ProcessLine(".x $HSCODE/hsfit/LoadHSFit.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    Weights
%%%%%%%%%%%%%%%%%%%%%%%%%    Bins
FFFFFFFFFFFFFFFFFFFFFFFFF RooHSEventsPDF

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

FFFFFFFFFFFFFFFFFFFFFFFFF RooHSEventsHistPDF
FFFFFFFFFFFFFFFFFFFFFFFFF HSMCMC
FFFFFFFFFFFFFFFFFFFFFFFFF HSRooFit
FFFFFFFFFFFFFFFFFFFFFFFFF HSsPlot


Create the sPlot class. All output will be saved in directory "out2"

In [2]:
  HSsPlot* RF=new HSsPlot("SF");
  RF->SetOutDir("out2/");
 

Load variables, 
Fit variable

        MesonMass = M(2K) discriminatory variable used to assess signal and background 
Cut variables (just used to filter events)

        Eg = tagged photon energy, coherent peak region 1.7-1.9 GeV
        MLP = result of MultiLayerPerceptron classifier > 0.98
        Topo = which final state topolgies to use, pK+ and pK-
        MesonCosPhi = CoM production angle of the $\phi$


In [3]:
 ///////////////////////////////Load Variables
  RF->LoadVariable("MesonMass[0.99,1.05]");//should be same name as variable in tree 
  RF->LoadAuxVars("Eg[1.7,1.9]");//Not to be fitted but limits applied to dataset
  RF->LoadAuxVars("MLP[0.98,1]");//Not to be fitted but limits applied to dataset
  RF->LoadAuxVars("Topo[0.5,2.5]");//Not to be fitted but limits applied to dataset
  RF->LoadAuxVars("MesonCosTh[-0.5,0.5]");//Not to be fitted but limits applied to dataset

Define the signal MesonMass PDF

Use custom RooHSEventsPDF class to base PDF shape on simulated data.
Additional parameters will be constrained in fit :

        alpha = Gaussian convolution width:  >0, MLV = 0, sigma = 0.1/5 GeV
        off = MesonMass peak offset: MLV = 0, sigma 0.1/10 GeV
        scale = MesonMass scale factor : MLV= 1, sigma 0.1/5, limits 0.9 and 1.1

In [4]:
 RF->Factory("RooHSEventsHistPDF::Phi(MesonMass,alpha[0,0,0.1],off[0,-0.05,0.05],scale[1,0.9,1.1])");
 RooHSEventsHistPDF* pdf=dynamic_cast<RooHSEventsHistPDF*>(RF->GetWorkSpace()->pdf("Phi"));

RooGaussian::AlphaConstrPhi[ x=alpha mean=0 sigma=0.02 ] = 1
RooGaussian::OffConstrPhi[ x=off mean=0 sigma=0.01 ] = 1
RooGaussian::ScConstrPhi[ x=scale mean=1 sigma=0.02 ] = 1
COPYNG 0x7fabea77e700 0
COPYNG 0x7fabeb107c00 0


Give the simulated data tree to the PDF

In [5]:
auto sfile=new TFile("/work/g8dump/hsexamples/converttestmvamlpinc2_sim.root");
auto stree=(TTree*)sfile->Get("newtree");
pdf->SetEvTree(stree,RF->GetCut());

MesonMass
0


Info in <HS::FIT::RooHSEventsHistPDF::RooHSEventsPDF::SetEvTree>:  with name newtree and cut MesonMass>=0.990000&&MesonMass<=1.050000&&Eg>=1.700000&&Eg<=1.900000&&MLP>=0.980000&&MLP<=1.000000&&Topo>=0.500000&&Topo<=2.500000&&MesonCosTh>=-0.500000&&MesonCosTh<=0.500000


tree entries 266366 1
      RooHSEventsHistPDF::CreateHistPdf()  
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(hmc_model_MesonMassPhi): fit range of variable offMesonMass expanded to nearest bin boundaries: [0.936667,1.10333] --> [0.936667,1.10333]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(hmc_model_MesonMassPhi): fit range of variable Valpha expanded to nearest bin boundaries: [0,0.1] --> [-0.0001,0.1001]


Register signal PDF to the Extended Maximimum Likelihood fit

In [6]:
 RF->LoadSpeciesPDF("Phi",1);

Apply Gaussian constraints to the signal PDF

In [7]:
  RF->AddGausConstraint(pdf->AlphaConstraint());
  RF->AddGausConstraint(pdf->OffConstraint());
  RF->AddGausConstraint(pdf->ScaleConstraint());

[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooGaussian::AlphaConstrPhi
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooConstVar::0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooConstVar::0.02
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooGaussian::OffConstrPhi
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooConstVar::0.01
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooGaussian::ScConstrPhi
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooConstVar::1


Register a 2 parameter Chebychev polynomial as the background PDF

In [8]:
 RF->Factory("Chebychev::BG(MesonMass,{a0[0.15,-1,1],a1[-0.25,-1,1]})");
 RF->LoadSpeciesPDF("BG");

Apply additional filter to the real data, but not the simulated data. Here the linear polarisation is not simulated and so we can not apply these cuts on polarisation state and degree to simulated data.

Load the data to be fitted. Here we take the output of the MLP finalstate analysis

In [9]:
RF->LoadAuxVars("Pol[0,1]");//Not to be fitted but limits applied to dataset
RF->LoadAuxVars("PolState[Polp=1,Polm=-1]");//Load a category
auto dfile=new TFile("/work/g8dump/hsexamples/converttestmvamlpinc2b.root");
auto dtree=(TTree*)dfile->Get("newtree");
RF->SetIDBranchName("UID");
RF->LoadDataSet(dtree);

[#1] INFO:Eval -- RooAbsReal::attachToTree(Topo) TTree Int_t branch Topo will be converted to double precision
[#1] INFO:DataHandling -- RooAbsCategory::attachToTree(PolState) TTree branch PolState will be interpreted as category index
[#1] INFO:Eval -- RooTreeDataStore::loadValues(newtree) Ignored 18464 out of range events
HSRooFit::LoadDataSet(TTree*) Print dataset for newtree
RooDataSet::newtree[MesonMass,Eg,MLP,Topo,MesonCosTh,Pol,PolState,UID] = 16314 entries


Turn on ROOT javascript magic for plotting

In [10]:
%jsroot on

Now just perform a single Maximum Likelihhod fit with Minuit2 minimiser. And draw the result

In [11]:
RF->RunAFit(1);

HSRooFit::TotalPDF() 0x7fabe8028fc0 0
RooAddPdf::SFTotalPDF[ Yld_Phi * Phi + Yld_BG * BG ] = 1.18398e+07
HSRooFit::TotalPDF() SFTotalPDF0
COPYNG 0x7fabeb0e7020 0x7fabea5aeac0
COPYNG 0x7fabebc31340 0x7fabebcce030
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) importing RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) using existing copy of HS::FIT::RooHSEventsHistPDF::Phi for import of RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) using existing copy of RooRealVar::MesonMass for import of RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) using existing copy of RooRealVar::off for import of RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) using existing copy of RooRealVar::scale for import of RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling -- RooWorkspace::import(wHS) using existing copy of RooRealVar::alpha for import of RooAddPdf::SFTotalPDF0
[#1] INFO:ObjectHandling

Info in <HS::FIT::HSsPlot::HSRooFit::Fit()>:  Starting


COPYNG 0x7fabeb0e7020 0x7fabea5aeac0
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_SFTotalPDF0_FOR_OBS_Eg:MLP:MesonCosTh:MesonMass:Pol:PolState:Topo:UID with 3 entries
[#1] INFO:Minization --  Including the following contraint terms in minimization: (AlphaConstrPhi,OffConstrPhi,ScConstrPhi)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_SFTotalPDF0_newtree_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimization -- RooAbsOptTestStatistic::optimizeConstantTerms(nll_SFTotalPDF0_newtree) WARNING Cache-and-track optimization (Optimize level 2) is only available for datasets implement in terms of RooVectorDataStore - ignoring this option for current dataset
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE    

Info in <HS::FIT::HSsPlot::HSsPlot::sPlot()>:  about to start


[#1] INFO:Minization -- createNLL picked up cached consraints from workspace with 3 entries
[#1] INFO:Minization --  Including the following contraint terms in minimization: (AlphaConstrPhi,OffConstrPhi,ScConstrPhi)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_SFTotalPDF0_newtree_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimization -- RooAbsOptTestStatistic::optimizeConstantTerms(nll_SFTotalPDF0_newtree) WARNING Cache-and-track optimization (Optimize level 2) is only available for datasets implement in terms of RooVectorDataStore - ignoring this option for current dataset
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (Phi,BG)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(SFTotalPDF0) Calculating sum-of-weights-squared correction matrix for covariance matrix
[#1] INFO:Minization -- RooMin

Info in <HS::FIT::HSsPlot::HSsPlot::FitAndStudy>: Save to out2//ResultsSF


Weights HSsWeights contains 16314 events associated file is out2//WeightsSF.root 
ID branch name : UID
Species are : 
BG
Phi
The first ten entries are :
271861 -0.299598 1.29956 
275878 -0.586851 1.58681 
277955 1.66024 -0.660263 
285811 -0.505197 1.50516 
291832 -0.594136 1.59409 
299296 1.22637 -0.226392 
300094 -0.576001 1.57596 
301113 -0.632388 1.63235 
305798 1.71078 -0.710797 
309577 1.49803 -0.498051 
Weights::BuildIndex 16314
Done
Weights::SortWeights() reordering trees 0x7fabcf75d640


Warning in <TTreeFormula::DefinedVariable>: Casting to primary types like "Long64_t" is not supported yet


Weights::SortWeights() reordering trees
Weights::SortWeights() entries 16314 16314
Weights::SortWeights Print new ordering
Weights HSsWeights contains 16314 events associated file is out2//WeightsSF.root 
ID branch name : UID
Species are : 
BG
Phi
The first ten entries are :
271861 -0.299598 1.29956 
275878 -0.586851 1.58681 
277955 1.66024 -0.660263 
285811 -0.505197 1.50516 
291832 -0.594136 1.59409 
299296 1.22637 -0.226392 
300094 -0.576001 1.57596 
301113 -0.632388 1.63235 
305798 1.71078 -0.710797 
309577 1.49803 -0.498051 
void Weights::Save() 0x7fabcf75d640
0x7fabcf6721a0 0x7fabebc67060
 Weights::Save() out2//WeightsSF.root 
******************************************************************************
*Tree    :HSsWeights_W: Tree weights for each species                          *
*Entries :    16314 : Total =          263309 bytes  File  Size =     244454 *
*        :          : Tree compression factor =   1.05                       *
*****************************************

In [12]:
((TCanvas*)RF->GetPlots()->At(1))->Draw()